In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pylab 
import statistics as stats
import scipy.stats as scipy
from pylab import rcParams
import statsmodels.api as sm
from sklearn import linear_model
import os

## Assignment 2: Weather Model

In [36]:
weather_df.columns

Index(['date', 'summary', 'preciptype', 'temperature', 'apparenttemperature',
       'humidity', 'windspeed', 'windbearing', 'visibility', 'loudcover',
       'pressure', 'dailysummary', 'humidity & windspeed',
       'humidity_and_windspeed'],
      dtype='object')

In [25]:
y_val = weather_df['apparenttemperature'] - weather_df['temperature']

In [27]:
# let's set up our model
X = weather_df[['humidity', 'windspeed']]
Y= y_val

In [30]:
# Bust out the OLS 
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Wed, 03 Jul 2019   Prob (F-statistic):               0.00
Time:                        16:02:22   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Questions: 
1. Are the estimated coefficients statistically signifigant?
Looking at all of the P-values (after interacting with the t-test) above I see that they are all 0.000, which is less than 0.01, meaning they are all statistically signifigant.


2. Are the signs of the estimated coefficients in line with your previous expectations?
My two features (humidity and windspeed) have negative coefficients which mean that as the Y value (apparent temp. minus temp.) increases the features decrease. This totally makes sense cognitively for windspeed but not for humidity. I live in Texas where it is both hot and humid, so this seems odd to me.


3. What are the relations between the target and explanatory variables?
As the target increases the X values decrease. So, as the target value increases by 1 points the windspeed decreases by 0.12 points and the humidity decreases by 3.03 points.

In [33]:
# Now we need to create a variable for the interaction of humidity and windspeed.
weather_df['humidity_and_windspeed'] = weather_df.humidity * weather_df.windspeed

In [34]:
# Re-running OLS with new variable included.
# let's set up our model
X = weather_df[['humidity', 'windspeed', 'humidity_and_windspeed']]
Y= y_val

In [35]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Wed, 03 Jul 2019   Prob (F-statistic):               0.00
Time:                        16:17:24   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      0.0839      0.033      2.511      0.012       0.018       0.149
humidity                   0.1775      0.043      4.133      0.000       0.093       0.262
windspeed                  0.0905      0.002     36.797      0.000       0.086       0.095
humidity_and_windspeed    -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Questions:
* Are the coefficients statistically signifigicant?
Yes, all three features (including the new interaction variable) have a p-value lower than 0.1 so they are statistically significant.

* Did my signs for humidity and windspeed change?
Yes! Both features now have positive coefficients. 


* What do the coefficients mean?
(Borrowed from solution - need to break down into English!)
According to the model, 1 point increase in the humidity and windspeed results in 0.18 and 0.09 point increases in the target respectively.
According to the model, the coefficient of the interaction term is -0.30. We can interpret it as follows. Given a windspeed level, 1 point increase in humidity results in 0.18 - 0.30 X windspeed point increase in the target. This means that the increase in the target is lower for high values of windspeed than for low values of windspeed. So, the windspeed mitigates the effect of humidity increase on the target. Similarly for a given humidity level, 1 point increase in the windspeed results in 0.09 - 0.30 X humidity point increase in the target. So, the humidity also mitigates the effect of windspeed on the target.

In [37]:
# let's set up our model
X = weather_df[['humidity', 'windspeed', 'visibility']]
Y= y_val

In [38]:
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                 1.401e+04
Date:                Wed, 03 Jul 2019   Prob (F-statistic):               0.00
Time:                        16:54:00   Log-Likelihood:            -1.6938e+05
No. Observations:               96453   AIC:                         3.388e+05
Df Residuals:                   96449   BIC:                         3.388e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5756      0.028     56.605      0.000       1.521       1.630
humidity      -2.6066      0.025   -102.784      0.000      -2.656      -2.557
windspeed     -0.1199      0.001   -179.014      0.000      -0.121      -0.119
visibility     0.0540      0.001     46.614      0.000       0.052       0.056
==============================================================================
Omnibus:                     3833.895   Durbin-Watson:                   0.282
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4584.022
Skew:                          -0.459   Prob(JB):                         0.00
Kurtosis:                       3.545   Cond. No.                         131.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Check Point 19.5: Evaluating Performance

#### Questions:
1. When X is humidity and windspeed: look at your R-Squared and Adjusted R-Squared. What are their values? Are they satisfactory? Why or why not?

On this model our R-squared and adjusted R-squared are both 0.288 or that our model explains about 28.8% of the variance in our target variable. This is a really low R-squared value and means that our model needs a lot of work before we could actually use it for predictions.

2. When X includes interaction feature: What is the new R-Squared? Does this model improve on the previous one?

For the second model our R-squared and adjusted R-squared value are both 0.341 meaning that our model explains 34.1% of the variation in the target variabe. This is an improvement on the previous scores, but still isn't good enough to use for predictions. 

3. Add visibility to the first X. Did R-Squared increase? What about adjusted R-squared? Which one is more useful?

Both R-squared and adjusted R-squared increased (0.304 & 0.303, respectively). Adjusted R-squared is always better to use because it accounts for the change in the number of features and is typically more accurate. 

4. Choose the best of your 3 models using AIC and BIC scores. Explain.

On all three models the AIC and BIC had the same value, so I am looking for the model with the lowest of these scores. Model 2 which adds in the interaction of humidity and windspeed as a feature has a AIC score of 3.334 x10^5 which is the lowest of all three. 